# Transformada Discreta de Fourier

In [91]:
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt

# Calcular a Matriz DFT

In [92]:
def dftmatrix(N: int, real: bool=True):
    """
        Devolve a matriz DFT de tamanho `N` por `N`. Se `real == True` então devolve apenas as primeiras `N//2 + 1` linhas da matriz preenchidas, pois são essas as linhas necessárias para obter os coeficientes de um vetor de números reais.
    """

    # Matriz
    W = np.ones((N, N), dtype=np.complex128)

    # Frequência fundamental
    omega = np.exp(-2 * np.pi * 1j / N)

    # Primeira linha com elementos diferentes de 1
    linha = np.array([omega ** j for j in range(0, N)])

    # Construir a matriz
    if real:
        for i in range(0, N//2 + 1):
            W[i, :] = linha ** i

    else:
        for i in range(0, N):
            W[i, :] = linha ** i
    
    return W

# Calcular a Matriz IDFT

In [93]:
def idftmatrix(N: int):
    """
        Devolve a matriz IDFT de tamanho `N` por `N`. Se `real == True` então devolve apenas as primeiras `N//2 + 1` linhas da matriz preenchidas, pois são essas as linhas necessárias para obter os coeficientes de um vetor de números reais.
    """

    # Matriz
    W = np.ones((N, N), dtype=np.complex128)

    # Frequência fundamental
    omega = np.exp(2 * np.pi * 1j / N)

    # Primeira linha com elementos diferentes de 1
    linha = np.array([omega ** j for j in range(0, N)])
    
    # Construir a matriz
    for i in range(0, N):
        W[i, :] = linha ** i
    
    return W / N

# Fazer a Transformada

In [94]:
def dft(yn: np.ndarray, W: np.ndarray, real: bool=True) -> np.ndarray:
    """
        Devolve os `N = len(yn)` primeiros coeficientes da Transformada de Fourier Discreta da sequência de números `yn` dada a matriz DFT N por N `W`. Se `real == True` então assume que os números são reais e usa isso para calcular apenas metade dos coeficientes.
    """

    # Determinar N
    N = len(yn)

    # Determinar os coeficientes
    if real:
        # Calcular os coeficientes por definição
        ck = np.zeros(N, dtype=np.complex128)
        ck[:(N//2) + 1] = (W @ yn)[:(N//2) + 1]

        # Usar o facto de termos números reais para obter logo a outra metade dos coeficientes
        for k in range(1, N//2 + 1):
            ck[N - k] = np.conjugate(ck[k])

        return ck
    
    else:
        # Calcular os coeficientes por definição
        return W @ yn

# Fazer a Transformada Inversa

In [95]:
def idft(ck: np.ndarray, iW: np.ndarray) -> np.ndarray:
    """
        Devolve os `N = len(ck)` valores obtidos ao fazer a Transformada de Fourier Discreta Inversa da sequência de números `ck` dada a matriz IDFT `iW`.
    """

    # Guardar os coeficientes
    N = len(ck)

    return iW @ ck

# Testar as Funções

## Construir as Matrizes

In [96]:
# Parâmetro
N = 4

W = dftmatrix(N, real=False)
print(np.array_str(W, precision=2, suppress_small=True))

[[ 1.+0.j  1.+0.j  1.+0.j  1.+0.j]
 [ 1.+0.j  0.-1.j -1.-0.j -0.+1.j]
 [ 1.+0.j -1.-0.j  1.+0.j -1.-0.j]
 [ 1.+0.j -0.+1.j -1.-0.j  0.-1.j]]


In [97]:
print(f"Largest difference is: {np.max(np.abs(np.linalg.inv(W) - idftmatrix(N)))}")

Largest difference is: 1.1985848907555554e-16


## Fazer a DFT

In [98]:
def f(x: np.ndarray) -> np.ndarray:
    return x


N = 10
tt = np.linspace(0, 1, N)
yy = f(tt)

isreal = True

Vamos verificar que `dft(yn) = np.dft(yy)`.

In [99]:
ck = dft(yy, dftmatrix(N), real=isreal)
cknp = np.fft.fft(yy)

print(f"Diferença máxima foi de {np.max(np.abs(ck - cknp)):.3e}")

Diferença máxima foi de 1.355e-15


Vamos verificar que `dft(yn) = np.dft(yy)`.

In [100]:
ck = idft(yy, idftmatrix(N))
cknp = np.fft.ifft(yy)

print(f"Diferença máxima foi de {np.max(np.abs(ck - cknp)):.3e}")

Diferença máxima foi de 6.777e-16


Vamos verificar que `idft(dft(yn)) = yy`.

In [101]:
ck = dft(yy, dftmatrix(N))
yy2 = idft(ck, idftmatrix(N))

print(f"\nDiferença máxima foi de {np.max(np.abs(yy2 - yy)):.3e}")


Diferença máxima foi de 1.848e-15


Podemos comparar este último resultado com o erro obtido usando `numpy`.

In [102]:
ck = np.fft.fft(yy)
yy2 = np.fft.ifft(ck)

print(f"\nDiferença máxima foi de {np.max(np.abs(yy2 - yy)):.3e}")


Diferença máxima foi de 1.675e-16
